In [8]:
from google.cloud import storage
import os
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [9]:
import numpy as np

In [10]:
df = pd.read_csv('gs://data_tql/WikiSQL/wikisql.csv')

In [11]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    blob_names = []
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names
    
def list_blobs_pd(bucket_name, folder_name):
    """List all files in given COS directory."""       
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    blob_time = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)
        blob_time.append(blob.time_created)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size, blob_time)), columns=['filePath', 'size', 'timeStamp'])    
    return blobs_df

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from COS bucket."""
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [18]:
bucket_name = 'data_tql'
all_blobs = list_blobs(bucket_name, 'wikisqliteDB/dev')


In [20]:
all_blobs[2]

'wikisqliteDB/dev/table_1_10015132_11.sqlite'

In [22]:
FILE_NAME = 'table_1_10015132_11.sqlite'
BUCKET_NAME = "data_tql"
FOLDER_NAME = "wikisqliteDB/dev"

storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob(f"{FOLDER_NAME}/{FILE_NAME}")

# Download the SQLite file to a local file
local_file_name = FILE_NAME
blob.download_to_filename(local_file_name)

In [24]:
conn = sqlite3.connect(f'table_1_10015132_11.sqlite')
cursor = conn.cursor()

In [27]:
df.loc[1,'sql']

'SELECT  col4 FROM table WHERE col5 = "New series began in June 2011"'

In [33]:
df1 = cursor.execute(f'SELECT col4 FROM table_1_10015132_11')

In [34]:
df1.fetchall()

[('1999-2000',),
 ('2002-03',),
 ('1996-97',),
 ('1996',),
 ('2002-03',),
 ('1996-97',),
 ('2012-present',)]

In [35]:
conn.close()